In [1]:
pip install moviepy whisper transformers


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pycaption


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pytesseract


Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from moviepy.editor import VideoFileClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from pydub import AudioSegment, silence
import whisper
import cv2
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load the Whisper model
logger.info("Loading Whisper model...")
model = whisper.load_model("base")  # Load the default model
logger.info("Model loaded.")

def detect_silence(audio_path, silence_thresh=-40, min_silence_len=500):
    audio = AudioSegment.from_file(audio_path)
    logger.info("Analyzing audio for silence...")
    silent_ranges = silence.detect_silence(
        audio, 
        min_silence_len=min_silence_len,
        silence_thresh=silence_thresh
    )
    silent_ranges = [(start / 1000, end / 1000) for start, end in silent_ranges]  # Convert to seconds
    return silent_ranges

def get_non_silent_segments(silent_segments, video_duration):
    non_silent_segments = []
    previous_end = 0
    for start, end in silent_segments:
        if start > previous_end:
            non_silent_segments.append((previous_end, start))
        previous_end = end
    if previous_end < video_duration:
        non_silent_segments.append((previous_end, video_duration))
    return non_silent_segments

def cut_video_segment(video_path, start_time, end_time, output_path):
    logger.info(f"Cutting video segment: start_time={start_time}, end_time={end_time}, output_path={output_path}")
    try:
        ffmpeg_extract_subclip(video_path, start_time, end_time, targetname=output_path)
        logger.info("Video segment cut.")
    except Exception as e:
        logger.error(f"Error cutting video segment: {e}")

def convert_video_to_audio(video_path, audio_path):
    try:
        video_clip = VideoFileClip(video_path)
        video_clip.audio.write_audiofile(audio_path)
        video_clip.close()
    except Exception as e:
        logger.error(f"Error converting video to audio: {e}")

def transcribe_audio(audio_path):
    try:
        result = model.transcribe(audio_path)
        return result['text']
    except Exception as e:
        logger.error(f"Error transcribing audio: {e}")
        return ""

def overlay_text_on_video(video_path, output_path, text):
    try:
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            font_scale = 3
            thickness = 10
            color = (255, 255, 255)
            text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)[0]
            text_x = int((width - text_size[0]) / 2)
            text_y = height - 100
            cv2.putText(frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness, cv2.LINE_AA)

            out.write(frame)

        cap.release()
        out.release()
    except Exception as e:
        logger.error(f"Error overlaying text on video: {e}")

def convert_video_to_gif(input_video_path, output_gif_path):
    try:
        clip = VideoFileClip(input_video_path)
        clip.write_gif(output_gif_path)
        clip.close()
    except Exception as e:
        logger.error(f"Error converting video to GIF: {e}")

def process_video(video_path, output_dir, silence_thresh=-40, min_silence_len=500):
    audio_path = "temp_audio.wav"
    
    logger.info(f"Extracting audio from video: {video_path}")
    try:
        video = VideoFileClip(video_path)
        video.audio.write_audiofile(audio_path)
        video.close()
    except Exception as e:
        logger.error(f"Error extracting audio from video: {e}")
        return []
    
    try:
        silent_segments = detect_silence(audio_path, silence_thresh, min_silence_len)
        logger.info(f"Silent segments detected: {silent_segments}")
    except Exception as e:
        logger.error(f"Error detecting silence: {e}")
        return []
    
    os.remove(audio_path)
    
    video_duration = VideoFileClip(video_path).duration  # Reload video to get duration
    non_silent_segments = get_non_silent_segments(silent_segments, video_duration)
    logger.info(f"Non-silent segments: {non_silent_segments}")
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    output_files = []
    for i, (start_time, end_time) in enumerate(non_silent_segments):
        video_segment_path = f"{output_dir}/segment_{i}.mp4"
        cut_video_segment(video_path, start_time, end_time, video_segment_path)
        
        segment_audio_path = f"{output_dir}/segment_{i}_audio.wav"
        convert_video_to_audio(video_segment_path, segment_audio_path)
        transcription = transcribe_audio(segment_audio_path)
        video_with_text_path = f"{output_dir}/segment_{i}_with_text.mp4"
        overlay_text_on_video(video_segment_path, video_with_text_path, transcription)
        
        gif_path = f"{output_dir}/segment_{i}.gif"
        convert_video_to_gif(video_with_text_path, gif_path)
        
        try:
            os.remove(video_segment_path)
            os.remove(segment_audio_path)
            os.remove(video_with_text_path)
        except Exception as e:
            logger.error(f"Error removing intermediate files: {e}")
        
        output_files.append(gif_path)
    
    return output_files

# Example usage
video_path = "C:/Users/91636/video.mp4"  # Provide the correct path to your video file
output_dir = "output_videoss"

try:
    output_files = process_video(video_path, output_dir)
    for file in output_files:
        logger.info(f"Generated GIF with captions: {file}")
except Exception as e:
    logger.error(f"Error occurred: {e}")


INFO:__main__:Loading Whisper model...
INFO:__main__:Model loaded.
INFO:__main__:Extracting audio from video: C:/Users/91636/video.mp4


MoviePy - Writing audio in temp_audio.wav


INFO:__main__:Analyzing audio for silence...                          


MoviePy - Done.


INFO:__main__:Silent segments detected: [(0.0, 1.082), (2.252, 4.391), (6.444, 7.643), (14.151, 15.065), (16.92, 17.982), (21.325, 22.15), (22.796, 23.723), (27.295, 28.187), (28.633, 29.551), (31.092, 33.466), (37.01, 41.081), (45.047, 45.584), (50.268, 51.368), (54.112, 55.815), (56.472, 62.41), (63.313, 65.816), (66.712, 67.779), (67.814, 68.945), (69.721, 72.159), (72.662, 74.507), (76.038, 78.756), (79.394, 85.969), (86.847, 88.229), (89.23, 90.161), (91.143, 92.154), (93.238, 94.223), (95.19, 96.478), (99.295, 99.952), (99.982, 100.901), (102.155, 103.383), (104.115, 105.487), (108.968, 109.764), (115.608, 117.17), (117.886, 118.623), (119.434, 122.89), (124.281, 125.2), (126.728, 127.856), (130.984, 131.749), (135.865, 137.522), (139.887, 141.073), (143.446, 144.441), (148.748, 149.301), (152.976, 153.653), (154.668, 155.959), (160.682, 161.597), (163.004, 163.662), (166.645, 167.835), (169.962, 170.885), (173.832, 177.179), (177.197, 178.113), (178.649, 179.73)]
INFO:__main__:N

...

  warnings.warn("Warning: in file %s, "%(self.filename)+

ERROR:__main__:Error converting video to audio: MoviePy error: failed to read the first frame of video file output_videoss/segment_48.mp4. That might mean that the file is corrupted. That may also mean that you are using a deprecated version of FFMPEG. On Ubuntu/Debian for instance the version in the repos is deprecated. Please update to a recent version from the website.
ERROR:__main__:Error transcribing audio: Failed to load audio: ffmpeg version N-115847-gab8f7030bc-20240617 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (crosstool-NG 1.26.0.65_ecc5e41)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --enable-shared --disable-static --disable-w32threads --enable-pthreads --enable-iconv --enable-libxml2 --enable-zlib --enable-libfreetype --enabl

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
MoviePy - Writing audio in output_videoss/segment_49_audio.wav


chunk:   0%|          | 0/59 [00:00<?, ?it/s, now=None]WARNING:py.warnings:C:\Users\91636\anaconda3\conda\Lib\site-packages\moviepy\audio\io\readers.py:197: UserWarning: Error in file output_videoss/segment_49.mp4, At time t=0.00-0.05 seconds, indices wanted: -34910--32916, but len(buffer)=23965
index -34910 is out of bounds for axis 0 with size 23965
  warnings.warn("Error in file %s, "%(self.filename)+

ERROR:__main__:Error converting video to audio: index -34910 is out of bounds for axis 0 with size 23965


MoviePy - Building file output_videoss/segment_49.gif with imageio.


ERROR:__main__:Error removing intermediate files: [WinError 32] The process cannot access the file because it is being used by another process: 'output_videoss/segment_49.mp4'
INFO:__main__:Generated GIF with captions: output_videoss/segment_0.gif
INFO:__main__:Generated GIF with captions: output_videoss/segment_1.gif
INFO:__main__:Generated GIF with captions: output_videoss/segment_2.gif
INFO:__main__:Generated GIF with captions: output_videoss/segment_3.gif
INFO:__main__:Generated GIF with captions: output_videoss/segment_4.gif
INFO:__main__:Generated GIF with captions: output_videoss/segment_5.gif
INFO:__main__:Generated GIF with captions: output_videoss/segment_6.gif
INFO:__main__:Generated GIF with captions: output_videoss/segment_7.gif
INFO:__main__:Generated GIF with captions: output_videoss/segment_8.gif
INFO:__main__:Generated GIF with captions: output_videoss/segment_9.gif
INFO:__main__:Generated GIF with captions: output_videoss/segment_10.gif
INFO:__main__:Generated GIF wit